In [12]:
#Import packages
import numpy as np
import scipy as sp
import matplotlib as ml
import os
import sys
flopypth = os.path.join('..', '..', 'flopy')
if flopypth not in sys.path:
    sys.path.append(flopypth)
import flopy
import flopy.utils.binaryfile as bf
import subprocess
import matplotlib.pyplot as plt
import shutil
from scipy.stats import uniform
import scipy.io as io
import datetime
from scipy.stats import uniform
from pyDOE import *

In [3]:
# Write function to generate range of inputs for hk, vka, pumping rate in each well using latin hypercube sampling
def genParamSamples(hk_min, hk_max, vka_min, vka_max, numWells, pump_min, pump_max, time_min, time_max, sampleSize):
    # Generate list of all paramters
    params_in_sample = ['hk', 'vka', 'time']
    for n in range(numWells):
        params_in_sample.append('pump_rate_' + str(n + 1))

    # Generate LHS samples
    numParam = np.size(params_in_sample)
    lhd = lhs(numParam, samples=sampleSize)

    # Generate arrays of hk and vka
    loc = hk_min
    scale = hk_max - hk_min
    hk = uniform(loc=loc, scale=scale).ppf(lhd[:, 0])
    loc = vka_min
    scale = vka_max - vka_min
    vka = uniform(loc=loc, scale=scale).ppf(lhd[:, 1])

    # Generate arrays of time
    loc = time_min
    scale = time_max - time_min
    time = uniform(loc=loc, scale=scale).ppf(lhd[:, 2])

    # Generate arrays of pumping rate
    pump = np.zeros([numWells, sampleSize])
    loc = pump_min
    scale = pump_max - pump_min
    for n in range(numWells):
        pump[n, :] = uniform(loc=loc, scale=scale).ppf(lhd[:, 3 + n])

    # Combine to form paramSample
    pumpSplit = np.split(pump, numWells)
    param_sample = np.stack([hk, vka, time])
    for i in range(numWells):
        param_sample = np.append(param_sample, pumpSplit[i], axis=0)

    # Create dictionary with samples for each parameter
    params = dict(zip(params_in_sample, param_sample))

    return params

In [5]:
# Plot settings
plotContours = True
plotHydrograph = True
modflowSilent = False

# Delete modflow files after use?
deleteFiles = True

# Save output?
saveOutput = True

# Parameter sample inputs
hk_min = 1.e-3
hk_max = 1.
vka_min = 1.e-3
vka_max = 1.e-2
numWells = 2
pump_min = -50000.
pump_max = -3000.
sampleSize = 5

# Fixed Parameter Definitions
# Model domain and grid definition
Lx = 1000.
Ly = 1000.
ztop = 1000.
zbot = 0.
nlay = 1
nrow = 50
ncol = 50
delr = Lx / ncol
delc = Ly / nrow
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1)
sy = 2.5e-1
ss = 4.e-7
laytyp = 1  # 1 = unconfined, 0 = confined
hdry = 0    # dry cell head set to this number
mxiter = 300
hclose = 1e-1

# Variables for the BAS package
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)    # Not sure if GW uses 0 or 1
strt = 1000 * np.ones((nlay, nrow, ncol), dtype=np.float32)     # Starting head

# Time step parameters
nper = 1    # number of stress periods
perlen_max = 3000     # length of stress period
perlen_min = 10
nstp = 500      # Number of time steps per stress period
steady = [False]

# Well locations
wpt1 = (0, 20-1, 37-1)
wpt2 = (0, 37-1, 18-1)

# Output control
spd = {(0, 0): ['print head', 'save head']}

In [7]:
# Get variable inputs
samples = genParamSamples(hk_min=hk_min, hk_max=hk_max,vka_min=vka_min, vka_max=vka_max, numWells=numWells,
                          pump_min=pump_min, pump_max=-pump_max, time_min=perlen_min, time_max=perlen_max, sampleSize=sampleSize)

# Define output parameters for each run
modflow_success = []
head_object = []
head_data1 = np.zeros([sampleSize, nstp])
head_data2 = np.zeros([sampleSize, nstp])
timeSeries = np.zeros([sampleSize, nstp])

# Get date and setup saving
datetimeStr = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2017-06-28 11:52:31'

In [8]:
i = 1

In [13]:
# Name model this run
model_name = 'model' + str(i)

# Get pumping rate same and make stress period dictionary
pumping_rate1 = samples['pump_rate_1'][i]
pumping_rate2 = samples['pump_rate_2'][i]
wel_sp1 = [0, 19, 36, pumping_rate1] # Remember to use zero-based layer, row, column indices!
wel_sp2 = [0, 36, 17, pumping_rate2]
stress_period_data = {0: [wel_sp1, wel_sp2]}

# Get hydraulic conductivty sample
hk = samples['hk'][i]
vka = samples['vka'][i]

# Get perlen sample
perlen = samples['time'][i]

# Flopy objects
mf = flopy.modflow.Modflow(model_name, exe_name='./mf2005dbl')
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,
                               top=ztop, botm=botm[1:],
                               nper=nper, perlen=perlen, nstp=nstp, steady=steady)
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(mf, hk=hk, vka=vka, sy=sy, ss=ss, laytyp=laytyp, hdry=hdry)
pcg = flopy.modflow.ModflowPcg(mf, mxiter=mxiter, hclose=hclose)  # This has stuff like iterations for solver
wel = flopy.modflow.ModflowWel(mf, stress_period_data=stress_period_data)
oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd)

# Write the model input files
mf.write_input()

# Run the model
success, modflow_output = mf.run_model(silent=modflowSilent, pause=False, report=True)
modflow_success.append(success)

# Create MODFLOW output file if there was an error
if not success:
    file = open('modflow_output' + str(i) + '.txt', 'w')
    for n in modflow_output:
        file.write(n + '\n')
    raise Warning('MODFLOW did not terminate normally.')

Exception: The program ./mf2005dbl does not exist or is not executable.

In [15]:
import numpy as np
hk_min = 1.e-3
hk_max = 1.
hk_avg = np.mean([hk_max, hk_min])
hk_avg

0.50049999999999994